# Estimating TOD distributions

Estimates TOD distribution for each TOD category for both school and departure time. 

TO DO: 
 - Keep adding more distributions to fit [avoid distributions with performance issues a.k.a too slow to run]
 - Generate validation grpahs [Data vs fitted distributions]

In [1]:
from collections import OrderedDict
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
from urbansim_templates.models import SmallMultinomialLogitStep
import orca
import os; os.chdir('../')
import warnings; warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels as sm
import matplotlib
import matplotlib.pyplot as plt
import time
from functools import reduce

import pickle
import dill

%matplotlib inline

## Define best distribution functions

In [2]:
'''
Code reference: 
https://stackoverflow.com/questions/6620471/fitting-empirical-distribution-to-theoretical-ones-with-scipy-python
'''

def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Distributions to check
#     DISTRIBUTIONS = [        
#         st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,
#         st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,
#         st.foldcauchy,st.foldnorm,st.frechet_r,st.frechet_l,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,
#         st.genextreme,st.gausshyper,st.gamma,st.gengamma,st.genhalflogistic,st.gilbrat,st.gompertz,st.gumbel_r,
#         st.gumbel_l,st.halfcauchy,st.halflogistic,st.halfnorm,st.halfgennorm,st.hypsecant,st.invgamma,st.invgauss,
#         st.invweibull,st.johnsonsb,st.johnsonsu,st.ksone,st.kstwobign,st.laplace,st.levy,st.levy_l,st.levy_stable,
#         st.logistic,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
#         st.nct,st.norm,st.pareto,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,st.rdist,st.reciprocal,
#         st.rayleigh,st.rice,st.recipinvgauss,st.semicircular,st.t,st.triang,st.truncexpon,st.truncnorm,st.tukeylambda,
#         st.uniform,st.vonmises,st.vonmises_line,st.wald,st.weibull_min,st.weibull_max,st.wrapcauchy
#     ]
    DISTRIBUTIONS = [
        st.chi,st.chi2,st.cosine,
        st.dgamma,st.expon,st.f,
        st.gamma,
        st.gumbel_l,st.halfnorm,st.halfgennorm,
        st.logistic,
        st.norm,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,
        st.uniform
    ]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)

def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

## Loading and cleaning data

In [3]:
#Loading data
trips = pd.read_csv('/home/emma/ual_model_workspace/spring-2019-models/notebooks-emma/HStrips_031219.csv', index_col = 0)

#select people who make both home-school and school-home trips:
tripsII = trips.groupby('HHPER').filter(lambda x: len(x) == 2)

#make sure all home-school trip rows are listed first
tripsIII = tripsII.sort_values(['HHPER','origin'])

#move school-home trip info up into home-school trip rows
tripsIII['school_dwell'] = tripsIII.groupby('HHPER', group_keys=False).origin_dwell.shift(-1)
tripsIII['school_ST'] = tripsIII.groupby('HHPER', group_keys=False).origin_ST.shift(-1)
tripsIII['SH_trip_ST'] = tripsIII.groupby('HHPER', group_keys=False).origin_ET.shift(-1)
tripsIII['SH_trip_ET'] = tripsIII.groupby('HHPER', group_keys=False).trip_ET.shift(-1)
tripsIII['SH_TT'] = tripsIII.groupby('HHPER', group_keys=False).TT.shift(-1)
tripsIII['SH_mode'] = tripsIII.groupby('HHPER', group_keys=False).MODE.shift(-1)

tripsIII = tripsIII.groupby('HHPER').first().reset_index()

tripsIII.rename(columns = {'origin_dwell':'home_dwell','origin_ST':'home_ST','origin_ET':'HS_trip_ST',
                           'trip_ET':'HS_trip_ET','TT':'HS_TT','MODE':'HS_mode','TOD':'HS_TOD'},inplace = True)

#School arrival time TOD category
tripsIII['School_arrival_TOD'] = pd.cut(tripsIII.HS_trip_ET, 
                             np.array([0, 3.0, 7.75, 8.5, 9.5, 15.0, 24.0]), 
                             labels = [6,1,2,3,4,5]).replace(6,5).astype(int)

#School departure time TOD category
tripsIII['School_departure_TOD'] = pd.cut(tripsIII.SH_trip_ST, 
                             np.array([0, 10, 12, 15, 17, 20, 24]), 
                             labels = [6,1,2,3,4,5]).replace(6, 5).astype(int)

trips_final = tripsIII

## Estimation

In [154]:
# Getting series for each TOD category
TOD_cat = [1,2,3,4,5]

TOD_series = dict()
for x in range(1,6):
    name_arrival = 'TOD_arrival_'+str(x)
    name_departure = 'TOD_departure_'+str(x)
    TOD_series[name_arrival] = trips_final.HS_trip_ET[trips_final.School_arrival_TOD == x]
    TOD_series[name_departure] = trips_final.SH_trip_ST[trips_final.School_departure_TOD == x]    

In [164]:
# Estimate distributions for each series 
TOD_best_distributions = dict()

for k,v in TOD_series.items(): 
    best_fit = best_fit_distribution(v, bins=200, ax=None)
    TOD_best_distributions[k] = best_fit

# best_fit_distribution(TOD_series['TOD_arrival_1'], bins=200, ax=None)

## Saving .plk file

In [169]:
#Creating a pkl file
file_Name = "/home/juan/activitysynth/activitysynth/configs/TOD_school_distributions.pkl"
fileObject = open(file_Name,'wb') 
dill.dump(TOD_best_distributions,fileObject)   
fileObject.close()

In [170]:
#Testing pkl file
fileObject = open(file_Name,'rb')  
# load the object from the file into var b
b = pickle.load(fileObject)  
b

{'TOD_arrival_1': ('pearson3',
  (-2.1741709726462837, 6.910093584993785, 0.9130500736229582)),
 'TOD_departure_1': ('pearson3',
  (-2.334661440450379, 11.505235230760526, 0.5775541144183648)),
 'TOD_arrival_2': ('chi',
  (1.2722603822487915, 7.764701020800063, 0.3379469030503246)),
 'TOD_departure_2': ('pearson3',
  (-2.4947923251836954, 14.44289891242401, 0.6949257588180353)),
 'TOD_arrival_3': ('chi',
  (1.1267933578858025, 8.515530267168568, 0.4567291677046388)),
 'TOD_departure_3': ('f',
  (6.140448179796783,
   5.12860067404247,
   14.95076976922487,
   0.5142603319944937)),
 'TOD_arrival_4': ('chi',
  (0.9276387555111918, 9.516666666666666, 2.5362791520006303)),
 'TOD_departure_4': ('powerlognorm',
  (143.27166513533655,
   2.0807126677365115,
   16.98060055651859,
   167.9898861447187)),
 'TOD_arrival_5': ('gumbel_l', (18.029208063456373, 1.0931053355051947)),
 'TOD_departure_5': ('dgamma',
  (0.6570762778899916, 20.999999999999996, 4.472119433950127))}